In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as sp
import os as os
import multiprocessing
from joblib import Parallel, delayed
from time import time as ti
from skimage.restoration import denoise_wavelet
import pickle
#import CoreFunctions as cf
import sys
import random
import psutil
import gc

In [ ]:
DataFolder = '/sciclone/scr10/dchendrickson01/Recordings2/'
DataFolder = '/scratch/Recordings2/'

DateString = 'Good'
LastGoodModel = 40

TIME_STEPS = 1200
Skips = 600
RollSize = 50

LastSuccesfull = 40

tic = ti()
start = tic

MemoryProtection = False


In [ ]:
%matplotlib inline

In [ ]:
FilesPerRun = 5
ConcurrentFiles = 5

In [ ]:
def RollingStdDevFaster(RawData, SmoothData, RollSize = 25):

    Diffs = RawData - SmoothData
    del RawData, SmoothData
    
    Sqs = Diffs * Diffs
    del Diffs
    
    Sqs = Sqs.tolist() 
    Sqs.extend(np.zeros(RollSize))
    mSqs = np.matrix(Sqs)
    
    for i in range(RollSize):
        Sqs.insert(0, Sqs.pop())
        mSqs = np.concatenate((np.matrix(Sqs),mSqs))
    
    sVect = mSqs.sum(axis=0)
    eVect = (mSqs!=0).sum(axis=0)
    del mSqs, Sqs
    
    VarVect = sVect / eVect
    StdDevs = np.sqrt(VarVect)
    return np.asarray(StdDevs[:-RollSize].T)

def SquelchPattern(DataSet, StallRange = 5000, SquelchLevel = 0.02, verbose = False):
    
    SquelchSignal = np.ones(len(DataSet))
    if verbose:
        print(len(SquelchSignal))
        
    for i in range(len(DataSet)-2*StallRange):
        if np.average(DataSet[i:i+StallRange]) < SquelchLevel:
            SquelchSignal[i+StallRange]=0

    return SquelchSignal

def split_MorS(original_list, ones_list,MorS,SegIndexes = False):
    # modified split_list_by_ones function to instead split by the zeros.
    #
    #
    # Created with Bing AI support
    #  1st request: "python split list into chunks based on value"
    #  2nd request: "I want to split the list based on the values in a second list.  Second list is all 1s and 0s.  I want all 0s removed, and each set of consequtive ones as its own item"
    #  3rd request: "That is close.  Here is an example of the two lists, and what I would want returned: original_list = [1, 2, 3, 8, 7, 4, 5, 6, 4, 7, 8, 9]
    #                ones_list =     [1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1]
    #                return: [[1, 2, 3, 8], [4, 5, 6], [8,9]]"
    #
    #This is the function that was created and seems to work on the short lists, going to use for long lists
    
    result_sublists = []
    SegsEnds = []
    sublist = []
    ii = 0
    
    if MorS == 0:
        for val, is_one in zip(original_list, ones_list):
            if not is_one:
                sublist.append(val)
            elif sublist:
                result_sublists.append(sublist)
                SegsEnds.append(ii)
                sublist = []
            ii+=1
    elif MorS == 1:
        for val, is_one in zip(original_list, ones_list):
            if is_one:
                sublist.append(val)
            elif sublist:
                result_sublists.append(sublist)
                SegsEnds.append(ii)
                sublist = []
            ii+=1
    
    # Add the last sublist (if any)
    if sublist:
        result_sublists.append(sublist)
        SegsEnds.append(ii)

    if SegIndexes:
        return result_sublists, SegsEnds
    else:
        return result_sublists

def split_list_by_ones(original_list, ones_list):
    # modified split_list_by_ones function to instead split by the zeros.
    #
    #
    # Created with Bing AI support
    #  1st request: "python split list into chunks based on value"
    #  2nd request: "I want to split the list based on the values in a second list.  Second list is all 1s and 0s.  I want all 0s removed, and each set of consequtive ones as its own item"
    #  3rd request: "That is close.  Here is an example of the two lists, and what I would want returned: original_list = [1, 2, 3, 8, 7, 4, 5, 6, 4, 7, 8, 9]
    #                ones_list =     [1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1]
    #                return: [[1, 2, 3, 8], [4, 5, 6], [8,9]]"
    #
    #This is the function that was created and seems to work on the short lists, going to use for long lists
    
    result_sublists = []
    sublist = []

    for val, is_one in zip(original_list, ones_list):
        if is_one:
            sublist.append(val)
        elif sublist:
            result_sublists.append(sublist)
            sublist = []

    # Add the last sublist (if any)
    if sublist:
        result_sublists.append(sublist)

    return result_sublists

# Generated training sequences for use in the model.
def create_sequences(values, time_steps=TIME_STEPS, skips = Skips, IndexEnd = 0):
    output = []
    ends = []
    for i in range(int((len(values) - time_steps + skips)/skips)):
        output.append(values[i*skips : (i*skips + time_steps)])
        ends.append(IndexEnd - len(values) + i*skips + time_steps)
    if IndexEnd ==0:
        return np.stack(output)
    else:
        return np.stack(output), ends

In [ ]:
def getVelocity(Acceleration, Timestamps = 0.003, Squelch = [], corrected = 0):
    velocity = np.zeros(len(Acceleration))
    
    Acceleration -= np.average(Acceleration)
    
    if len(Timestamps) == 1:
        dTime = np.ones(len(Acceleration),dtype=float) * Timestamps
    elif len(Timestamps) == len(Acceleration):
        dTime = np.zeros(len(Timestamps), dtype=float)
        dTime[0]=1
        for i in range(len(Timestamps)-1):
            j = i+1
            if Timestamps[j] > Timestamps[i]:
                dTime[j]=Timestamps[j]-Timestamps[i]
            else:
                dTime[j]=Timestamps[j]-Timestamps[i]+10000.0
        dTime /= 10000.0

    velocity[0] = Acceleration[0] * (dTime[0])

    for i in range(len(Acceleration)-1):
        j = i + 1
        if corrected ==2:
            if Squelch[j]==0:
                velocity[j]=0
            else:
                velocity[j] = velocity[i] + Acceleration[j] * dTime[j]                
        else:
            velocity[j] = velocity[i] + Acceleration[j] * dTime[j]

    if corrected == 1:
        PointVairance = velocity[-1:] / len(velocity)
        for i in range(len(velocity)):
            velocity[i] -=  PointVairance * i
    
    velocity *= 9.81

    return velocity

In [ ]:
%matplotlib inline
def runFile(file, verbose = False, small = False, index=0, start=ti()):
    noise = verbose
    dataset = pd.read_csv(DataFolder+file, delimiter =",", header=None, engine='python',on_bad_lines='skip')
    
    
    dataset = dataset.rename(columns={0:"Day"})
    dataset = dataset.rename(columns={1:"Second"})
    dataset = dataset.rename(columns={2:"FracSec"})
    dataset = dataset.rename(columns={3:"p"})
    dataset = dataset.rename(columns={4:"h"})
    dataset = dataset.rename(columns={5:"v"})
    dataset = dataset.rename(columns={6:"Sensor"})
    
    #dataset['Second'].replace('',0)
    #dataset['FracSec'].replace('',0)
    #dataset.replace([np.nan, np.inf, -np.inf],0,inplace=True)
    
    #dataset[['Day','Second']] = dataset[['Day','Second']].apply(lambda x: x.astype(int).astype(str).str.zfill(6))
    #dataset[['FracSec']] = dataset[['FracSec']].apply(lambda x: x.astype(int).astype(str).str.zfill(4))
    
    #dataset["timestamp"] = pd.to_datetime(dataset.Day+dataset.Second+dataset.FracSec,format='%y%m%d%H%M%S%f')
    #dataset["timestamps"] = dataset["timestamp"]
    
    dataset["p"] = dataset.p - np.average(dataset.p)
    dataset["h"] = dataset.h - np.average(dataset.h)
    dataset["v"] = dataset.v - np.average(dataset.v)
    #dataset["r"] = np.sqrt(dataset.p**2 + dataset.h**2 + dataset.v**2)
    
    #dataset.index = dataset.timestamp
    
    dataset["SmoothP"] = denoise_wavelet(dataset.p, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
    dataset["SmoothH"] = denoise_wavelet(dataset.h, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
    dataset["SmoothV"] = denoise_wavelet(dataset.v, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
    #dataset["SmoothR"] = denoise_wavelet(dataset.r, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
    
    
    RawData = dataset.v
    SmoothData = dataset.SmoothV
    RollSize = 25
    
    Diffs = RawData - SmoothData
    
    Sqs = Diffs * Diffs
    
    Sqs = Sqs.tolist() 
    
    Sqs.extend(np.zeros(RollSize))
    
    mSqs = np.matrix(Sqs)
    
    for i in range(RollSize):
        Sqs.insert(0, Sqs.pop())
        mSqs = np.concatenate((np.matrix(Sqs),mSqs))
    
    sVect = mSqs.sum(axis=0)
    eVect = (mSqs!=0).sum(axis=0)
    
    VarVect = sVect / eVect
    
    StdDevs = np.sqrt(VarVect)
    
    StdDevsZ = np.asarray(StdDevs)
    
    StdDevsZ=np.append(StdDevsZ,[0])
    
    StdDevsZ = np.asarray(StdDevsZ.T[:len(dataset.p)])
    
    print("Size StdDevsZ", ti()-start, np.shape(StdDevsZ))
    
    StdDevsZ = np.nan_to_num(StdDevsZ)
    
    StdDevsZ[StdDevsZ == np.inf] = 0
    StdDevsZ[StdDevsZ == -np.inf] = 0
    
    SmoothDevZ = denoise_wavelet(StdDevsZ, method='VisuShrink', mode='soft', wavelet='sym2', rescale_sigma='True')
    
    print("denoise 1", ti()-start, np.shape(StdDevsZ))
    
    SmoothDevZ[np.isnan(SmoothDevZ)]=0
    
    Max = np.max(SmoothDevZ)
    
    buckets = int(Max / 0.005) + 1
    bins = np.linspace(0,buckets*0.005,buckets+1)
    counts, bins = np.histogram(SmoothDevZ,bins=bins)
    
    CummCount = 0
    HalfWay = 0
    for i in range(len(counts)):
        CummCount += counts[i]
        if CummCount / len(SmoothDevZ) >= 0.5:
            if HalfWay == 0:
                HalfWay = i
    
    SquelchLevel = bins[HalfWay] 

    fig = plt.figure()
    plt.plot(SmoothDevZ)
    plt.show()    

    if SquelchLevel > .015:
        SL = 0.04
    else:
        SL = 0.02
    
    
    dataset["IsMoving"] = SquelchPattern(SmoothDevZ, 2000, SL, False)
    
    
    velocity = getVelocity(dataset.p, dataset.FracSec, dataset.IsMoving, 2)
    
    df_pr, SegEndr = split_MorS(dataset.p, dataset.IsMoving,1,SegIndexes = True)
    df_hr = split_MorS(dataset.h, dataset.IsMoving,1)
    df_vr = split_MorS(dataset.v, dataset.IsMoving,1)
    
    df_ps, SegEnds = split_MorS(dataset.SmoothP, dataset.IsMoving,0,SegIndexes = True)
    df_hs = split_MorS(dataset.SmoothH, dataset.IsMoving,0)
    df_vs = split_MorS(dataset.SmoothV, dataset.IsMoving,0)
    
    SmoothMove = []
    for i in range(len(df_pr)):
        if len(df_pr[i])>TIME_STEPS:
            tempP,tempE=create_sequences(df_pr[i],IndexEnd=SegEndr[i])
            tempH=create_sequences(df_hr[i])
            tempV=create_sequences(df_vr[i])
            for j in range(len(tempP)):
                SmoothMove.append([np.vstack((tempP[j],tempH[j],tempV[j])),tempE[j],file]) #,df_rs[i])))
    
    SmoothStop = []
    for i in range(len(df_ps)):
        if len(df_ps[i])>TIME_STEPS:
            tempP,tempE=create_sequences(df_ps[i],IndexEnd=SegEnds[i])
            tempH=create_sequences(df_hs[i])
            tempV=create_sequences(df_vs[i])
            for j in range(len(tempP)):
                SmoothStop.append([np.vstack((tempP[j],tempH[j],tempV[j])),tempE[j],file]) #,df_rs[i])))
    
    return SmoothMove, SmoothStop, velocity, SmoothDevZ
    #else:
    #    return ['fail','fail']
        
        #if verbose:
        #    print('Data normalized', ti()-start)

        #return df_p, df_h, df_v, df_r, df_rp, df_rh, df_rv, df_rr


In [ ]:
import CoreFunctions as cf

In [ ]:
def runWrapper(file_path, verbose=True, small=False, index=0, start=ti()):
    #try:
    rtrn = runFile(file_path, verbose, small, index, start)
    return rtrn
    #except Exception as e:
    #    with open('BadInputs.text', 'a') as bad_file:
    #        bad_file.write(file_path + '\n')
    #    return np.zeros((10, 10, 3))

In [ ]:
def PrintWrap(Mat):
    localPrints = []
    lenm = np.shape(Mat)[1]
    slices = int(lenm/TIME_STEPS)
    for i in range(slices):
        temp = (cf.makeMPFast(Mat[:3,i*TIME_STEPS:(i+1)*TIME_STEPS], wvt = 'sym4', scales = 32, spacer = 2, title = ''))
        localPrints.append(temp.astype(np.float32)/255.0)
    return localPrints

# Start Machine Learning
## Using Autoencoder with Kears and Tensorflow
cite: https://pyimagesearch.com/2020/02/17/autoencoders-with-keras-tensorflow-and-deep-learning/

In [ ]:
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np

In [ ]:
class ConvAutoencoder:
    @staticmethod
    def build(width, height, depth, filters=(32, 64), latentDim=24):
        inputShape = (height, width, depth)
        chanDim = -1
        inputs = Input(shape=inputShape)
        x = inputs

        for f in filters:
            x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)

        volumeSize = K.int_shape(x)
        print("Volume Size:", volumeSize)
        x = Flatten()(x)
        latent = Dense(latentDim)(x)

        encoder = Model(inputs, latent, name="encoder")

        latentInputs = Input(shape=(latentDim,))
        flattenedVolumeSize = int(np.prod(volumeSize[1:]))
        print("Flattened Volume Size:", flattenedVolumeSize)
        x = Dense(flattenedVolumeSize)(latentInputs)
        x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)

        for f in filters[::-1]:
            x = Conv2DTranspose(f, (3, 3), strides=2, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)

        x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
        outputs = Activation("sigmoid")(x)

        decoder = Model(latentInputs, outputs, name="decoder")
        autoencoder = Model(inputs, decoder(encoder(inputs)), name="autoencoder")

        return (encoder, decoder, autoencoder)


In [ ]:
# set the matplotlib backend so figures can be saved in the background
#import matplotlib
#matplotlib.use("Agg")
# import the necessary packages
#from pyimagesearch.convautoencoder import ConvAutoencoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
# construct the argument parse and parse the arguments

In [ ]:
import random
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
from tensorflow.keras.callbacks import Callback

In [ ]:
import keras
import re

In [ ]:
directory = '/scratch/models/'
target_file = f'3DFP_{DateString}_{str(LastGoodModel).zfill(3)}_good2_ae.keras'

ImageShape=[5,32,600,3]

encoder = load_model(directory+target_file[:-8]+'en.keras')
decoder = load_model(directory+target_file[:-8]+'de.keras')

autoencoder_input = Input(shape=(ImageShape[1], ImageShape[2], ImageShape[3]))

# Pass the input through the encoder and decoder
encoded_repr = encoder(autoencoder_input)
reconstructed = decoder(encoded_repr)

# Create the reassembled autoencoder model
autoencoder = Model(autoencoder_input, reconstructed)

In [ ]:
print(encoder.summary())
print(decoder.summary())
print(autoencoder.summary())
#if LastSuccesfull != 0:
#    print(reautoencoder.summary())

In [ ]:
def getTensors(file_list):

    AllDatas = Parallel(n_jobs=ConcurrentFiles)(delayed(runFile)(file_list[i], False, False, 0, ti()) for i in range(min(len(file_list),FilesPerRun)))

    Mats=[]
    StopIndex=[]
    Runs=[]
    MoveIndex=[]
    StopFiles=[]
    MoveFiles=[]

    for fileResponse in AllDatas:
        for Mat in fileResponse[0]:
            Mats.append(Mat[0])
            StopIndex.append(Mat[1])
            StopFiles.append(Mat[2])
        for Mat in fileResponse[1]:
            Runs.append(Mat[0])
            MoveIndex.append(Mat[1])
            MoveFiles.append(Mat[2])
        fig = plt.figure()
        plt.plot(fileResponse[3])
        plt.show()
        fig = plt.figure()
        plt.plot(fileResponse[2])
        plt.show()
        
            
    if MemoryProtection:
        del AllDatas
        print('RAM after AllData:', psutil.virtual_memory()[2], len(Mats))        

    AllPrints = Parallel(n_jobs=8)(delayed(PrintWrap)(Mat) for Mat in Mats)
    AllRuns = Parallel(n_jobs=8)(delayed(PrintWrap)(Mat) for Mat in Runs)
    
    if MemoryProtection:
        del Mats, Runs
        print('RAM after Keeps:', psutil.virtual_memory()[2])
    
    Prints = []
    Rprint = []    
    for group in AllPrints:
        for fprint in group:
            Prints.append(fprint[:, ::2, :])
    for group in AllRuns:
        for fprint in group:
            Rprint.append(fprint[:, ::2, :])
    
    if MemoryProtection:
        del AllPrints, AllRuns
    
    for i, image in enumerate(Prints):
        if not isinstance(image, np.ndarray):
            Prints[i] = np.array(image, dtype=np.float32)
        elif image.dtype != np.float32:
            Prints[i] = image.astype(np.float32)
    for i, image in enumerate(Rprint):
        if not isinstance(image, np.ndarray):
            Rprint[i] = np.array(image, dtype=np.float32)
        elif image.dtype != np.float32:
            Rprint[i] = image.astype(np.float32)
    
    # Stack the images into a single NumPy array
    prints_array = np.stack(Prints, axis=0)
    rprint_array = np.stack(Rprint, axis=0)
    
    if MemoryProtection:
        del Prints, Rprint
        print('RAM after Prints:', psutil.virtual_memory()[2])
    # Convert the NumPy array to a TensorFlow tensor
    trX = tf.convert_to_tensor(prints_array)
    trR = tf.convert_to_tensor(rprint_array)
    
    if MemoryProtection:
        del prints_array, rprint_array
    return trX, trR, fileResponse[3], fileResponse[2], MoveIndex, StopIndex, MoveFiles, StopFiles, Runs, Mats

In [ ]:
files=['230801 recording1.csv',
       '230802 recording1.csv',
       '230424 recording3.csv',
       '230426 recording3.csv',
       '230801 recording1.csv',
       '230802 recording1.csv',
       '230424 recording3.csv',
       '230426 recording3.csv',
       '230119 recording1.csv'
       ]

In [ ]:
%%time
Moves, Stops, StdDev, Velocity, MoveLoc, StopLoc, MoveFiles, StopFiles, MoveData, StopData = getTensors(files)

In [ ]:
len(Moves)

In [ ]:
len(Stops)

In [ ]:
MovePred = autoencoder.predict(Moves)
StopPred = autoencoder.predict(Stops)

In [ ]:
Diff_Move = Moves-MovePred
Diff_Stop = Stops-StopPred

In [ ]:
MSE_Move_Pixel = np.mean(Diff_Move**2,axis=3)
MSE_Stop_Pixel = np.mean(Diff_Stop**2,axis=3)

In [ ]:
MSE_Move_Time = np.mean(MSE_Move_Pixel,axis=1)
MSE_Stop_Time = np.mean(MSE_Stop_Pixel,axis=1)

In [ ]:
MSE_Move_Frame = np.mean(MSE_Move_Time,axis=1)
MSE_Stop_Frame = np.mean(MSE_Stop_Time,axis=1)
print('RAM after Predictions and Deltas:', psutil.virtual_memory()[2])

In [ ]:
np.shape(Diff_Move)

In [ ]:
np.shape(MSE_Move_Pixel)

In [ ]:
np.shape(MSE_Move_Time)

In [ ]:
np.shape(MSE_Move_Frame)

In [ ]:
MoveResult = np.mean(MSE_Move_Frame)
StopResult = np.mean(MSE_Stop_Frame)

In [ ]:
MoveResult

In [ ]:
StopResult

In [ ]:
RResultsS = []
RResultsA = []
RResultsM = []
RResultsAM = []
RResultsMS = []
for result in Diff_Move:
    RResultsS.append(np.sum(result))
    RResultsA.append(np.sum(np.abs(result)))
    RResultsM.append(np.mean(result))
    RResultsAM.append(np.mean(np.abs(result)))
    RResultsMS.append(np.mean(result**2))

In [ ]:
SResultsS = []
SResultsA = []
SResultsM = []
SResultsAM = []
SResultsMS = []
for result in Diff_Stop:
    SResultsS.append(np.sum(result))
    SResultsA.append(np.sum(np.abs(result)))
    SResultsM.append(np.mean(result))
    SResultsAM.append(np.mean(np.abs(result)))
    SResultsMS.append(np.mean(result**2))

In [ ]:
def filter_data(data, min = -np.inf, max = np.inf):
    data = np.array(data)
    data[np.isnan(data)] = 0
    data[np.isinf(data)] = 0
    mean = np.mean(data) 
    std_dev = np.std(data) 
    filtered_data = data[np.abs(data - mean) / std_dev < 2]
    filtered_data = filtered_data[filtered_data > min]
    filtered_data = filtered_data[filtered_data < max]

    return filtered_data

In [ ]:
plt.hist(filter_data(RResultsS), bins=20)
plt.xlabel("Sum Difs")
plt.ylabel("No of samples")
plt.show()
plt.hist((RResultsS), bins=20)
plt.xlabel("Sum Difs")
plt.ylabel("No of samples")
plt.show()

In [ ]:
plt.hist(filter_data(RResultsA,0,50000), bins=20)
plt.xlabel("Sum Abs Value Diffs")
plt.ylabel("No of samples")
plt.show()
plt.hist(RResultsA, bins=20)
plt.xlabel("Sum Abs Value Diffs")
plt.ylabel("No of samples")
plt.show()

In [ ]:
plt.hist(filter_data(RResultsAM,0,2), bins=20)
plt.xlabel("Mean Abs Val Difs")
plt.ylabel("No of samples")
plt.show()
plt.hist(RResultsAM, bins=20)
plt.xlabel("Sum Difs")
plt.ylabel("Mean Abs Val Difs")
plt.show()


In [ ]:
plt.hist((RResultsS), bins=20)
plt.xlabel("Sum Difs")
plt.ylabel("No of samples")
plt.show()

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,figsize=(8,3), dpi=300 )
ax0.hist((RResultsS), bins=20)
ax1.hist((SResultsS), bins=20)
plt.show()

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,figsize=(8,3), dpi=300 )
ax0.hist((RResultsA), bins=20)
ax1.hist((SResultsA), bins=20)
plt.show()

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,figsize=(8,3), dpi=300 )
ax0.hist((RResultsMS), bins=20)
ax1.hist((SResultsMS), bins=20)
plt.show()

In [ ]:
Diff_Move = Moves-MovePred

In [ ]:
for i in range(15):
    fig, (ax0, ax1, ax2) = plt.subplots(1, 3,figsize=(10,3), dpi=300 )
    ax0.imshow(Moves[i], origin='lower',aspect='auto')
    ax0.axis("off")
    ax1.imshow(MovePred[i], origin='lower',aspect='auto')
    ax1.axis("off")
    ax2.imshow(np.abs(Diff_Move[i]), origin='lower',aspect='auto')
    ax2.axis("off")
    plt.show()

In [ ]:
for i in range(15):
    fig, (ax0, ax1, ax2) = plt.subplots(1, 3,figsize=(10,3), dpi=300 )
    ax0.imshow(Stops[i], origin='lower',aspect='auto')
    ax0.axis("off")
    ax1.imshow(StopPred[i], origin='lower',aspect='auto')
    ax1.axis("off")
    ax2.imshow(np.abs(Diff_Stop[i]), origin='lower',aspect='auto')
    ax2.axis("off")
    plt.show()

In [ ]:
BigMoveError = np.argmax(RResultsMS)
fig, (ax0, ax1, ax2) = plt.subplots(1, 3,figsize=(10,3), dpi=300 )
ax0.imshow(Moves[BigMoveError], origin='lower',aspect='auto')
ax0.axis("off")
ax1.imshow(MovePred[BigMoveError], origin='lower',aspect='auto')
ax1.axis("off")
ax2.imshow(np.abs(Diff_Move[BigMoveError]), origin='lower',aspect='auto')
ax2.axis("off")
plt.show()
BigStopError = np.argmax(SResultsMS)
fig, (ax0, ax1, ax2) = plt.subplots(1, 3,figsize=(10,3), dpi=300 )
ax0.imshow(Stops[BigStopError], origin='lower',aspect='auto')
ax0.axis("off")
ax1.imshow(StopPred[BigStopError], origin='lower',aspect='auto')
ax1.axis("off")
ax2.imshow(np.abs(Diff_Stop[BigStopError]), origin='lower',aspect='auto')
ax2.axis("off")
plt.show()

In [ ]:
BigMoveError = np.argmin(RResultsMS)
fig, (ax0, ax1, ax2) = plt.subplots(1, 3,figsize=(10,3), dpi=300 )
ax0.imshow(Moves[BigMoveError], origin='lower',aspect='auto')
ax0.axis("off")
ax1.imshow(MovePred[BigMoveError], origin='lower',aspect='auto')
ax1.axis("off")
ax2.imshow(np.abs(Diff_Move[BigMoveError]), origin='lower',aspect='auto')
ax2.axis("off")
plt.show()
BigStopError = np.argmin(SResultsMS)
fig, (ax0, ax1, ax2) = plt.subplots(1, 3,figsize=(10,3), dpi=300 )
ax0.imshow(Stops[BigStopError], origin='lower',aspect='auto')
ax0.axis("off")
ax1.imshow(StopPred[BigStopError], origin='lower',aspect='auto')
ax1.axis("off")
ax2.imshow(np.abs(Diff_Stop[BigStopError]), origin='lower',aspect='auto')
ax2.axis("off")
plt.show()

MoveLoc, StopLoc, MoveFiles, StopFiles, MoveData, StopData

In [ ]:
print(MoveLoc[np.argmin(RResultsMS)],MoveFiles[np.argmin(RResultsMS)])

In [ ]:
fig = plt.figure()
plt.plot(StopData[np.argmin(RResultsMS)][0,:])
plt.plot(StopData[np.argmin(RResultsMS)][1,:])
plt.plot(StopData[np.argmin(RResultsMS)][2,:])
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(MoveData[np.argmin(SResultsMS)][0,:])
plt.plot(MoveData[np.argmin(SResultsMS)][1,:])
plt.plot(MoveData[np.argmin(SResultsMS)][2,:])
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(MoveData[np.argmax(SResultsMS)][0,:])
plt.plot(MoveData[np.argmax(SResultsMS)][1,:])
plt.plot(MoveData[np.argmax(SResultsMS)][2,:])
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(StopData[np.argmax(RResultsMS)][0,:])
plt.plot(StopData[np.argmax(RResultsMS)][1,:])
plt.plot(StopData[np.argmax(RResultsMS)][2,:])
plt.show()

In [ ]:
StopData = np.array(StopData)
StopSortIndexes=np.argsort(RResultsMS)
StopSorted=StopData[StopSortIndexes]

In [ ]:
for i in range(15):
    fig = plt.figure(figsize=(8,3),dpi=300)
    plt.plot(StopSorted[i][0,:],linewidth=0.5)
    plt.plot(StopSorted[i][1,:],linewidth=0.5)
    plt.plot(StopSorted[i][2,:],linewidth=0.5)
    plt.show()

In [ ]:
j = len(StopSorted)-1
for i in range(15):
    fig = plt.figure(figsize=(8,3),dpi=300)
    plt.plot(StopSorted[j-i][0,:],linewidth=0.5)
    plt.plot(StopSorted[j-i][1,:],linewidth=0.5)
    plt.plot(StopSorted[j-i][2,:],linewidth=0.5)
    plt.show()

In [ ]:
MoveData = np.array(MoveData)
MoveSortIndexes=np.argsort(SResultsMS)
MoveSorted=MoveData[MoveSortIndexes]

In [ ]:
for i in range(15):
    fig = plt.figure(figsize=(8,3),dpi=300)
    plt.plot(MoveSorted[i][0,:],linewidth=0.5)
    plt.plot(MoveSorted[i][1,:],linewidth=0.5)
    plt.plot(MoveSorted[i][2,:],linewidth=0.5)
    plt.show()

In [ ]:
j = len(MoveSorted)-1
for i in range(15):
    fig = plt.figure(figsize=(8,3),dpi=300)
    plt.plot(MoveSorted[j-i][0,:],linewidth=0.5)
    plt.plot(MoveSorted[j-i][1,:],linewidth=0.5)
    plt.plot(MoveSorted[j-i][2,:],linewidth=0.5)
    plt.show()